In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
import matplotlib
#matplotlib.use('Agg')
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim import models
from tqdm.notebook import tqdm

Using TensorFlow backend.


In [2]:
#embed_size = 50 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a comment to use
train = pd.read_csv("train_stance_pre.csv")
train_body = pd.read_csv("train_body_pre.csv")
list_sentences_train = train["Headline"].fillna("_na_").values
list_sentences_body = train_body["articleBody"]
#subm = pd.read_csv("sample_submission.csv")
#cat = pd.read_json("categories.json")
list_classes = ["agree","disagree","discuss","unrelated"]
#list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
tokenizer.fit_on_texts(list(list_sentences_body))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_body)
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
#print(X_t[0])
#print(len(X_t))
X_te = pad_sequences(list_tokenized_test, maxlen=300)

In [3]:
train.head()

,Body ID,Headline,agree,disagree,discuss,unrelated
0,712,police find mass grave with at least ' bodies'...,0.0,0.0,0.0,1.0
1,158,hundred of palestinian flee flood in gaza a is...,1.0,0.0,0.0,0.0
2,137,christian bale pas on role of steve job actor ...,0.0,0.0,0.0,1.0
3,1034,hbo and apple in talk for month apple tv strea...,0.0,0.0,0.0,1.0
4,1923,spider burrow through tourist's stomach and up...,0.0,1.0,0.0,0.0


In [4]:
X_tte = np.zeros((len(X_t),300),dtype = np.int32)
for i in range(len(train['Body ID'])):
    X_tte[i] = X_te[train_body['Body ID']==train['Body ID'][i]]

In [5]:
'''train_body['articleBody'][train_body['Body ID']==712]
 w = models.KeyedVectors.load_word2vec_format(
    './GoogleNews-vectors-negative300.bin', binary=True)
w['be']
embedding_matrix[tokenizer.word_index['be']]'''

"train_body['articleBody'][train_body['Body ID']==712]\n w = models.KeyedVectors.load_word2vec_format(\n    './GoogleNews-vectors-negative300.bin', binary=True)\nw['be']\nembedding_matrix[tokenizer.word_index['be']]"

In [6]:
len(np.where(y==1)[1])

49972

In [7]:
embed_size = 300 # how big is each word vector
#max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)
max_features = min(len(tokenizer.word_index),20000)
maxlen = 100 # max number of words in a comment to use
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.shape
    emb_layer = nn.Embedding(max_features, embed_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim
def loadGloveModel():
    print("Loading Glove Model")

    f = open('glove.6B.100d.txt','r')
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print(len(gloveModel)," words loaded!")
    return gloveModel
def loadGoogleModel():
    w = models.KeyedVectors.load_word2vec_format(
    './GoogleNews-vectors-negative300.bin', binary=True)
    return w
def get_embed(word_idx,max_features,embed_dim):
    #glove = loadGloveModel()
    glove = loadGoogleModel()
    #all_embs = np.stack(glove.to_numpy())
    #all_embs = np.stack(glove.values())
    #emb_mean,emb_std = all_embs.mean(), all_embs.std()
    emb_mean = 0
    emb_std = 0.6
    word_index = word_idx
    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_dim))
    #print("go into model")
    for word, i in word_index.items():
        if i >= max_features: continue
        try: 
            embedding_vector = glove[word]
        except:
            None
            #print('skip')
            #embedding_vector = np.random.normal(scale=0.6, size=(100, ))
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
    return embedding_matrix
embedding_matrix = get_embed(tokenizer.word_index,max_features,embed_size)

In [8]:
from torch.utils.data import Dataset
class LoadDataset(Dataset):
    def __init__(self, train_body,train_stance,y):
        self.body = train_body
        self.sentence = train_stance
        self.label = y
        #self.labels = train_label_lt
    def __getitem__(self, idx):
        body = self.body[idx]
        sentence = self.sentence[idx]
        y = self.label[idx]    
        #label = self.labels[idx]
        #filename = self.imgs[idx]
        return body,sentence,y
    def __len__(self):
        return len(self.sentence)

In [9]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import autograd
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
from torch.utils.data import SequentialSampler
from torch.utils.data import Sampler
import random

In [5]:
label_3_idx = sum([np.where(y==1)[1] == 3])
label_2_idx = sum([np.where(y==1)[1] == 2])
label_1_idx = sum([np.where(y==1)[1] == 1])
label_0_idx = sum([np.where(y==1)[1] == 0])
print(sum(label_1_idx))
label_0_idx = [idx  for idx,t in enumerate(label_0_idx) if t == 1]
label_1_idx = [idx  for idx,t in enumerate(label_1_idx) if t == 1]
label_2_idx = [idx  for idx,t in enumerate(label_2_idx) if t == 1]
label_3_idx = [idx  for idx,t in enumerate(label_3_idx) if t == 1]

840


In [11]:
'''class IISampler(Sampler):
    def __init__(self, y_,index0,index1,index2,index3,batch_size,drop_last):
        self.batch_size = batch_size
        self.sampler = y_
        self.drop_last = drop_last
        self.indence = index0
        self.indence1 = index1
        self.indence2 = index2
        self.indence3 = index3
    def __iter__(self):
        batch = []
        #for j in range(4):
        #batch.append(random.sample(self.indence,16))
        #batch.append(random.sample(self.indence1,16))
        #batch.append(random.sample(self.indence2,16))
        #batch.append(random.sample(self.indence3,16))
        idx = random.sample(self.indence,16)
        idx1 = random.sample(self.indence1,16)
        idx2 = random.sample(self.indence2,16)
        idx3 = random.sample(self.indence3,16)
        batch.append(idx)
        batch.append(idx1)
        batch.append(idx2)
        batch.append(idx3)
        batch = np.array(batch).reshape(self.batch_size)
        np.random.shuffle(batch)
        #print(batch)
        if len(batch) == self.batch_size:
            #return batch
            yield batch
            #batch = []
        #return batch
        #return batch
        #return (self.indices[i] for i in torch.randperm(len(self.indices)))

    def __len__(self):
        if self.drop_last:
            return len(self.sampler) // self.batch_size
        else:
            return (len(self.sampler) + self.batch_size - 1) // self.batch_size

print(len(a))
a=np.array([4,2,3,1])
remove_ixs=np.where(a>1)[0]
print(remove_ixs)
b=np.delete(a,remove_ixs)
print(b)'''

'class IISampler(Sampler):\n    def __init__(self, y_,index0,index1,index2,index3,batch_size,drop_last):\n        self.batch_size = batch_size\n        self.sampler = y_\n        self.drop_last = drop_last\n        self.indence = index0\n        self.indence1 = index1\n        self.indence2 = index2\n        self.indence3 = index3\n    def __iter__(self):\n        batch = []\n        #for j in range(4):\n        #batch.append(random.sample(self.indence,16))\n        #batch.append(random.sample(self.indence1,16))\n        #batch.append(random.sample(self.indence2,16))\n        #batch.append(random.sample(self.indence3,16))\n        idx = random.sample(self.indence,16)\n        idx1 = random.sample(self.indence1,16)\n        idx2 = random.sample(self.indence2,16)\n        idx3 = random.sample(self.indence3,16)\n        batch.append(idx)\n        batch.append(idx1)\n        batch.append(idx2)\n        batch.append(idx3)\n        batch = np.array(batch).reshape(self.batch_size)\n        np

In [12]:
#label_3_idx[0]
x3 = np.array(label_3_idx)
np.random.shuffle(x3)
a = x3[8001::]

In [13]:
batch_size = 64
X_ts = np.delete(X_t,a,axis=0)
X_ttes = np.delete(X_tte,a,axis=0)
train_X_sentence = torch.from_numpy(X_t)
train_X_sentence = train_X_sentence.view(-1, 100).type(torch.FloatTensor)
train_X_body = torch.from_numpy(X_tte)
train_X_body = train_X_body.view(-1, 300).type(torch.FloatTensor)
temp = np.where(y==1)[1]
#temp2 = np.delete(temp,a,axis=0)
y_ = torch.from_numpy(temp)
#ampler = IISampler(y_,label_0_idx,label_1_idx, label_2_idx ,label_3_idx,64,drop_last=True)
#test_dataset = Data.TensorDataset()
train_dataset = LoadDataset(train_X_body,train_X_sentence,y_)
train_loader = DataLoader(
    dataset = train_dataset,      
    batch_size = batch_size,      # mini batch size
    shuffle = True,           
    #sampler=ampler
    #num_workers=2,              
)

In [14]:
n_vocab = max_features
embedding_dim = 300
hidden_size = 256
num_class = 4
class DCNN(nn.Module):
    def __init__(self, weights_matrix):
        super(DCNN,self).__init__()
        weight = torch.FloatTensor(weights_matrix)
        #print(type(weight))
        self.embedding1 = nn.Embedding.from_pretrained(weight.cuda()) #
        self.embedding2 = nn.Embedding.from_pretrained(weight.cuda()) #.
        self.embedding1.weight.requires_grad = False
        self.embedding2.weight.requires_grad = False
        #self.embedding1 = nn.Embedding(n_vocab, embedding_dim)
        #self.embedding2 = nn.Embedding(n_vocab, embedding_dim)
        #self.embedding, num_embeddings, embedding_dim = create_emb_layer(weights_matrix, True)
        self.hidden_size = hidden_size
        self.embedding_dim = embedding_dim
        #self.num_layers = num_layers
        #self.gru = nn.GRU(embed_dim, self.hidden_size, num_layers=self.num_layers, batch_first=True,bidirectional=False)
        #self.conv1 = nn.Sequential(nn.Conv1d(in_channels= enbedding_dim,out_channels=))
        #self.conv1 = nn.Conv1d(self.hidden_size,64)
        #self.gru = nn.GRU(embedding_dim, self.hidden_size, num_layers, batch_first=True)
        self.conv0 = nn.Sequential(nn.Conv1d(in_channels=self.embedding_dim, 
                                        out_channels=self.hidden_size, 
                                        kernel_size=3,padding=3),
#                              nn.BatchNorm1d(num_features=config.feature_size), 
                              nn.ReLU(0.1),
                              nn.Dropout(0.5),
                              nn.MaxPool1d(kernel_size=2),
                              )
        self.conv1 = nn.Sequential(nn.Conv1d(in_channels=self.hidden_size, 
                                   out_channels=self.hidden_size, 
                                   kernel_size=3,padding=3), 
                                   nn.ReLU(0.1),
                                   nn.Dropout(0.5),
                                   nn.MaxPool1d(kernel_size=2),)
        self.conv2 = nn.Sequential(nn.Conv1d(in_channels=self.hidden_size, 
                                   out_channels=self.hidden_size*2, 
                                   kernel_size=3,padding=3), 
                                   nn.ReLU(0.1),
                                   nn.Dropout(0.5),
                                   nn.MaxPool1d(kernel_size=2),)
        self.conv3 = nn.Sequential(nn.Conv1d(in_channels=self.hidden_size*2, 
                                   out_channels=self.hidden_size*2, 
                                   kernel_size=3,padding=1), 
                                   nn.ReLU(0.1),
                                   nn.Dropout(0.5),
                                   #nn.MaxPool1d(kernel_size=2),
                                  )
        self.conv4 = nn.Sequential(nn.Conv1d(in_channels=self.hidden_size*2, 
                                   out_channels=self.hidden_size*3, 
                                   kernel_size=3,padding=1), 
                                   nn.ReLU(0.1),
                                   nn.Dropout(0.5),
                                   #nn.MaxPool1d(kernel_size=2),
                                  )
        self.conv0_1 = nn.Sequential(nn.Conv1d(in_channels=self.embedding_dim, 
                                        out_channels=self.hidden_size, 
                                        kernel_size=3,padding=3),
#                              nn.BatchNorm1d(num_features=config.feature_size), 
                              nn.ReLU(0.1),
                              nn.Dropout(0.5),
                              nn.MaxPool1d(kernel_size=2),
                              )
        self.conv1_1 = nn.Sequential(nn.Conv1d(in_channels=self.hidden_size, 
                                   out_channels=self.hidden_size, 
                                   kernel_size=3,padding=3), 
                                   nn.ReLU(0.1),
                                   nn.Dropout(0.5),
                                   nn.MaxPool1d(kernel_size=2),)
        self.conv2_1 = nn.Sequential(nn.Conv1d(in_channels=self.hidden_size, 
                                   out_channels=self.hidden_size*2, 
                                   kernel_size=3,padding=3), 
                                   nn.ReLU(0.1),
                                   nn.Dropout(0.5),
                                   nn.MaxPool1d(kernel_size=2),)
        self.conv3_1 = nn.Sequential(nn.Conv1d(in_channels=self.hidden_size*2, 
                                   out_channels=self.hidden_size*2, 
                                   kernel_size=3,padding=1), 
                                   nn.ReLU(0.1),
                                   nn.Dropout(0.5),
                                   #nn.MaxPool1d(kernel_size=2),
                                  )
        self.conv4_1 = nn.Sequential(nn.Conv1d(in_channels=self.hidden_size*2, 
                                   out_channels=self.hidden_size*3, 
                                   kernel_size=3,padding=1), 
                                   nn.ReLU(0.1),
                                   nn.Dropout(0.5),
                                   #nn.MaxPool1d(kernel_size=2),
                                  )
        self.final = nn.Sequential(nn.Linear(600,1024),
                                  #nn.Linear(1024,1024),
                                  nn.Linear(1024,num_class))
        self.final1  = nn.Sequential(#nn.Linear(43776,1024),
                                  nn.Linear(57,1024),
                                  #nn.Linear(1024,1024),
                                  nn.Linear(1024,num_class))
        self.body_lin = nn.Linear(41,200)
        self.head_lin = nn.Linear(16,200)
        self.body_k = nn.Linear(200,100)
        self.body_q = nn.Linear(200,100)
        self.head_k = nn.Linear(200,100)
        self.head_q = nn.Linear(200,100)
        self.softmax = nn.Softmax(dim=1)
        self.sigmoid = nn.Sigmoid()
    def forward(self,x,x1):
        bs= x.shape[0]
        embed_x = self.embedding1(x)
        #x , x1 = x
        #outputs = self.conv0(x)
        outputs = self.conv0(embed_x.permute(0, 2, 1))
        outputs = self.conv1(outputs)
        outputs = self.conv2(outputs)
        outputs = self.conv3(outputs)
        outputs = self.conv4(outputs)
        embed_x_1 = self.embedding2(x1)
        #outputs_1 = self.conv0_1(x1)
        outputs_1 = self.conv0_1(embed_x_1.permute(0, 2, 1))
        outputs_1 = self.conv1_1(outputs_1)
        outputs_1 = self.conv2_1(outputs_1)
        outputs_1 = self.conv3_1(outputs_1)
        outputs_1 = self.conv4_1(outputs_1)
        #outputs = self.final(outputs[:,-1])
        #head_vec = outputs_1.view(bs,-1)
        #body_vec = outputs.view(bs,-1)
        head_vec = self.max_over_time(outputs_1)
        body_vec = self.max_over_time(outputs)
        #feature_vec = torch.cat([head_vec,body_vec],1).view(bs,-1)
        #outputs = self.final1(feature_vec)
        #return self.softmax(outputs)
    
        head_vec1 = self.head_lin(head_vec).view(-1,1,200)
        body_vec1 = self.body_lin(body_vec).view(-1,1,200)
        head_vec2 = self.head_lin(head_vec).view(-1,1,200)
        body_vec2 = self.body_lin(body_vec).view(-1,1,200)
        body_vec3 = self.body_lin(body_vec).view(-1,1,200)
        head_vec3 = self.head_lin(head_vec).view(-1,1,200)
        v_vec = self.atten_vec(bs,3,200)
        #body_vec = body_vec1
        #head_vec = head_vec1
        body_vec = torch.cat([body_vec1,body_vec2,body_vec3],1)
        head_vec = torch.cat([head_vec1,head_vec2,head_vec3],1)
        score = torch.matmul(head_vec,body_vec.transpose(1,2))
        #body_k = self.body_k(body_vec).view(-1,1,100)
        #body_q = self.body_q(body_vec).view(-1,1,100)
        #head_k = self.head_k(head_vec).view(-1,1,100)
        #head_q = self.head_q(head_vec).view(-1,1,100)
        #score2 = torch.matmul(body_q,head_k.permute(0,2,1)).view(-1,1)
        #score1 = torch.matmul(head_q,body_k.permute(0,2,1)).view(-1,1)
        #scores = torch.cat([score1,score2],1)#([head_vec*scores[0],body_vec*scores[1]],1)
        
        scores = self.softmax(score)
        scores = torch.matmul(score,v_vec)
        #print(scores.shape)
        #feature_vec = torch.cat([head_vec,body_vec],1)
        feature_vec = scores.view(bs,-1)
        #print(feature_vec.shape)
        outputs = self.final(feature_vec)
        
        #outputs = self.softmax(outputs)
        outputs = self.sigmoid(outputs)
        #print(outputs.shape)
        return outputs
    def atten_vec(self,bs,head=3,size = 200):
        return autograd.Variable(torch.randn(bs,head,size)).to(device)
    def max_over_time(self,output):
        m = output.max(axis=1)[0]
        return m.reshape(m.shape[0],m.shape[1])

In [15]:
class Main_model():
    def __init__(self,model,train_loader,test_loader,LR,epoch,weight_decay,device): #,test_loader
        self.LR =LR
        self.epochs = epoch
        #print(device)
        self.model = model.to(device)
        self.weight_decay = weight_decay
        #self.optimizer = torch.optim.Adam(self.model.parameters(), lr = LR,weight_decay=self.weight_decay)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr = LR)
        #self.optimizer = torch.optim.Adam(self.model.parameters(), lr=LR, betas=(0.9, 0.99))
        self.criterion = nn.CrossEntropyLoss()
        #self. criterion = nn.BCELoss()
        #self.criterion = nn.MSELoss()
        self.train_loader = train_loader
        self.ans = []
        #self.test_loader = test_loader
    def forward(self):
        history_loss = []
        history_train_acc = []
        self.ans = []
        #history_test_acc = []
        for epoch in tqdm(range(self.epochs)):
            train_loss, train_acc = self.train(epoch)
            #test_loss, test_acc = self.test(epoch)
            history_loss.append(train_loss)
            history_train_acc.append(train_acc)
            #history_test_acc.append(test_acc)
            print(np.argmax(self.ans[-1],axis=1))
        return history_loss, history_train_acc, #history_test_acc
    def train(self,epoch):
        self.model.train()
        correct = 0
        train_loss = 0
        total = 0
        acc = 0
        #ans = []
        #mts=[]
        #k=0
        #for tzs in tqdm(range(50)):
        for idx,(body_data,sentence_data, target) in tqdm(enumerate(self.train_loader)): 
            if device!='cpu':
                #data = torch.LongTensor(data)

                body_data = body_data.type(torch.LongTensor).squeeze()
                sentence_data = sentence_data.type(torch.LongTensor).squeeze()
                target = target.type(torch.LongTensor).squeeze()
                body_data,sentence_data, target = body_data.to(device), sentence_data.to(device),target.to(device)
            self.optimizer.zero_grad()
            output = self.model(body_data,sentence_data)
            #print(output)
            #print(target)
            loss = self.criterion(output,target)
            loss.backward()
            self.optimizer.step() 
            train_loss +=loss

            pred = output.argmax(dim=1, keepdim=True)
            bs = body_data.shape[0]
            for i in range(bs):
                #corre = corr/k

                self.ans.append(output.detach().cpu().numpy())
                #print(output[i])
                #print(target[i])
                #if pred[i].item() == target[i].item(): #torch.max(target, 1)[1][i].item():
                if pred[i].item() == target[i].item():
                    correct+=1
        #correct1 = acc/total
        correct /= len(self.train_loader.dataset)
        train_loss /= len(self.train_loader)
        print('Epoch:', epoch,'|train loss:%.4f'%train_loss,'|train accuracy: %.4f'%correct)
        #mts.append([epoch,train_loss.item(),correct])
        return train_loss,correct

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
Models = DCNN(embedding_matrix)
LSTT = Main_model(Models,train_loader,_,0.005,500,1e-2,device)
history_loss, history_train_acc = LSTT.forward()


Epoch: 0 |train loss:1.4903 |train accuracy: 0.4405
[0 1 3 1 0 1 0 0 1 0 0 1 0 0 1 0 0 2 0 2 0 0 0 3 0 0 1 1 0 1 0 0 0 1 3 0 0
 2 0 2 3 2 1 1 0 1 0 0 1 0 0 2]



Epoch: 1 |train loss:1.4881 |train accuracy: 0.4426
[2 1 0 1 3 1 3 0 1 0 2 0 0 1 0 2 0 1 0 1 0 1 0 1 0 0 0 1 1 1 0 0 1 1 2 0 0
 0 2 0 2 2 1 0 2 0 0 0 1 0 3 0]



Epoch: 2 |train loss:1.4912 |train accuracy: 0.4415
[3 0 0 1 0 0 0 1 1 0 2 0 0 0 1 2 1 0 0 0 0 3 1 1 0 0 2 1 0 0 0 0 0 0 1 1 0
 1 0 0 0 1 0 0 0 3 2 0 2 0 2 1]



Epoch: 3 |train loss:1.4902 |train accuracy: 0.4392
[0 1 0 1 0 1 0 0 0 0 2 1 1 1 3 0 0 0 0 0 3 3 1 1 1 2 0 0 1 0 0 2 0 1 0 0 0
 1 2 0 1 1 0 1 1 0 0 3 0 0 1 2]



Epoch: 4 |train loss:1.4900 |train accuracy: 0.4397
[1 2 0 2 1 0 1 0 3 2 0 0 3 2 1 0 0 1 1 1 0 2 0 1 0 0 0 0 2 0 0 2 0 1 3 0 2
 0 2 2 0 3 1 1 0 1 1 0 0 0 0 2]


...


Epoch: 286 |train loss:1.4939 |train accuracy: 0.4356
[1 0 2 2 0 3 0 0 0 2 2 1 2 1 0 0 0 2 1 0 0 0 1 0 1 1 3 0 1 1 1 3 0 1 2 2 3
 0 1 1 0 0 0 0 1 0 0 1 1 0 2 0]



Epoch: 287 |train loss:1.4894 |train accuracy: 0.4429
[1 3 0 1 2 0 1 0 3 0 2 0 0 0 0 0 1 1 3 1 0 0 0 1 2 0 0 1 1 1 1 0 0 2 0 2 0
 0 1 0 0 1 0 3 0 1 1 2 0 2 1 0]



Epoch: 288 |train loss:1.4882 |train accuracy: 0.4416
[1 0 1 0 0 1 1 2 1 0 0 0 2 0 0 1 3 0 3 0 1 1 0 1 1 1 1 0 0 0 1 2 1 0 0 3 1
 0 0 3 0 1 0 0 0 0 2 1 1 0 0 1]



Epoch: 289 |train loss:1.4895 |train accuracy: 0.4412
[1 0 0 1 0 0 0 1 0 2 2 1 0 1 0 1 0 0 0 0 1 0 0 3 0 2 1 0 0 0 0 1 1 0 2 0 0
 2 1 1 2 0 3 0 1 0 1 1 2 1 1 0]



Epoch: 290 |train loss:1.4883 |train accuracy: 0.4433
[2 2 2 0 0 3 0 3 0 1 0 0 3 1 0 2 0 1 1 0 0 0 0 1 2 0 0 2 3 0 0 1 0 0 1 0 0
 3 0 0 0 1 1 0 0 1 0 1 0 0 0 0]


In [ ]:
np.argmax(LSTT.ans[500000],axis=1)

In [ ]:
len(LSTT.ans)